TEST_DATA

In [1]:
import pandas as pd

coupang_3p = pd.read_excel("./test_data/(전달용) 쿠팡_리뷰 20240223 복사본.xlsx", sheet_name="(RAW) 쿠팡_3P")
coupang_rocket = pd.read_excel("./test_data/(전달용) 쿠팡_리뷰 20240223 복사본.xlsx", sheet_name="(RAW) 쿠팡_로켓")

In [2]:
# 3P 데이터에서 "등록된 내용이 없습니다" 제외하고 컬럼명 변경
coupang_3p_cleaned = coupang_3p[coupang_3p['상품평 코멘트'] != '등록된 내용이 없습니다.'].rename(columns={'상품평 코멘트': '리뷰내용'})

# 로켓 데이터에서 "등록된 내용이 없습니다" 제외
coupang_rocket_cleaned = coupang_rocket[coupang_rocket['리뷰내용'] != '등록된 내용이 없습니다.']

# 정제된 데이터 합치기
combined_reviews = pd.concat([coupang_3p_cleaned, coupang_rocket_cleaned], ignore_index=True)

combined_reviews = combined_reviews.dropna(subset='리뷰내용')

# 결과 확인
print(f"정제 후 총 리뷰 수: {len(combined_reviews)}")
print(f"정제 후 3P 리뷰 수: {len(coupang_3p_cleaned)}")
print(f"정제 후 로켓 리뷰 수: {len(coupang_rocket_cleaned)}")

정제 후 총 리뷰 수: 3083
정제 후 3P 리뷰 수: 2060
정제 후 로켓 리뷰 수: 1053


In [3]:
test_data = combined_reviews['리뷰내용'][:1000]

In [4]:
# 줄바꿈으로 구분하여 합치기
combined_text_newline = '\n'.join(test_data)

### ABSA_LLM

In [7]:
# LLM 객체 생성
from langchain.chat_models import ChatOpenAI

from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [8]:
# 환경변수
from dotenv import load_dotenv

load_dotenv()

True

In [14]:
# LLM_1
ABSA_llm = ChatOpenAI(
    temperature=0.7,  # 창의성
    model_name="gpt-4o",  # 모델명
)

In [12]:
ABSA_template = """
    Define the sentiment elements as follows:
    
    − The 'aspect term' refers to a specific feature, attribute, or aspect of an item, product, or service that a user might comment on. If implicit, the aspect term can be 'null'.
    − The 'opinion term' reflects the sentiment or attitude toward a specific aspect. It can also be 'null' for implicit opinions.
    − The 'aspect category' is the general category to which the aspect belongs. Examples may include 'quality', 'pricing', 'availability', 'appearance', 'functionality', 'service', 'general', and other suitable categories as applicable across diverse domains.
    − The 'sentiment polarity' expresses whether the sentiment is 'positive', 'negative', or 'neutral'.

    Based on the provided definitions, identify and organize all sentiment elements from the following text by grouping them by sentiment polarity. 
    
    Please answer in Korean.

    Text to analyze:
    {text}
"""

In [15]:
# 프롬프트 생성
ABSA_prompt = ChatPromptTemplate.from_template(ABSA_template)

# 체인 생성 (프롬프트와 LLM 연결)
ABSA_chain = LLMChain(llm=ABSA_llm, prompt=ABSA_prompt)

# 체인을 통해 combined_reviews 텍스트로 실행
ABSA_result = ABSA_chain.run({"text": test_data})

print(ABSA_result)

**긍정적 감정 요소:**

1. 문서 0:
   - 측면 용어: 섭취
   - 의견 용어: 간편해서 좋아요
   - 측면 범주: 편리성
   - 감정 극성: 긍정적

   - 측면 용어: 배송
   - 의견 용어: 빠르게
   - 측면 범주: 서비스
   - 감정 극성: 긍정적

2. 문서 2:
   - 측면 용어: 관절
   - 의견 용어: 최고예요
   - 측면 범주: 품질
   - 감정 극성: 긍정적

3. 문서 4:
   - 측면 용어: 성분
   - 의견 용어: 좋아서
   - 측면 범주: 품질
   - 감정 극성: 긍정적

4. 문서 1012:
   - 측면 용어: 알약 크기
   - 의견 용어: 작아서 좋음
   - 측면 범주: 편리성
   - 감정 극성: 긍정적

   - 측면 용어: 목넘김
   - 의견 용어: 쉽네요
   - 측면 범주: 편리성
   - 감정 극성: 긍정적

5. 문서 1013:
   - 측면 용어: 가격
   - 의견 용어: 가성비 최고에요
   - 측면 범주: 가격
   - 감정 극성: 긍정적

6. 문서 1015:
   - 측면 용어: 알 크기
   - 의견 용어: 더 작고
   - 측면 범주: 편리성
   - 감정 극성: 긍정적

   - 측면 용어: 위생적
   - 의견 용어: 위생적
   - 측면 범주: 품질
   - 감정 극성: 긍정적

**부정적 감정 요소:**

1. 문서 2:
   - 측면 용어: 손목 관절
   - 의견 용어: 너무 안좋아요
   - 측면 범주: 건강
   - 감정 극성: 부정적

   - 측면 용어: 무릎
   - 의견 용어: 안좋은
   - 측면 범주: 건강
   - 감정 극성: 부정적

2. 문서 3:
   - 측면 용어: 무릎 관절
   - 의견 용어: 너무 안좋고 이상하다
   - 측면 범주: 건강
   - 감정 극성: 부정적

   - 측면 용어: 운동
   - 의견 용어: 더 아픈
   - 측면 범주: 건강
   - 감정 극성: 부정적

**중립적 감정 요소:**

해당 텍스트

In [12]:
ABSA_result = """

### 긍정적 요소
1. **측면 용어**: 가격, 품질, 배송  
   - **의견 용어**: 합리적, 만족스럽다, 좋다, 빠르다  
   - **측면 범주**: 가격, 품질, 편리성  
   - **감정 극성**: 긍정적  

2. **측면 용어**: 효과, 사용  
   - **의견 용어**: 기대 이상, 좋았다, 신뢰할 수 있다  
   - **측면 범주**: 효과, 신뢰성  
   - **감정 극성**: 긍정적  

3. **측면 용어**: 성능, 효능, 신뢰성  
   - **의견 용어**: 탁월하다, 믿음이 간다, 매우 만족한다  
   - **측면 범주**: 성능, 신뢰성  
   - **감정 극성**: 긍정적  

### 중립적 요소
1. **측면 용어**: 효과, 기대  
   - **의견 용어**: 기대하고 있다, 아직 효과는 모르겠다  
   - **측면 범주**: 효과, 기대  
   - **감정 극성**: 중립적  

2. **측면 용어**: 편리성, 사용  
   - **의견 용어**: 나쁘지는 않다, 아직 결정할 수 없다  
   - **측면 범주**: 편리성  
   - **감정 극성**: 중립적  

3. **측면 용어**: 품질, 기능  
   - **의견 용어**: 무난하다, 보통이다, 기대만큼이다  
   - **측면 범주**: 품질, 기능  
   - **감정 극성**: 중립적  

### 부정적 요소
1. **측면 용어**: 효과, 기대  
   - **의견 용어**: 기대만큼 효과가 없다, 효과가 미미하다  
   - **측면 범주**: 효과  
   - **감정 극성**: 부정적  

2. **측면 용어**: 가격, 품질  
   - **의견 용어**: 가격 대비 만족스럽지 않다, 별다른 효과가 없다  
   - **측면 범주**: 가격, 효과  
   - **감정 극성**: 부정적  

3. **측면 용어**: 사용, 편리성  
   - **의견 용어**: 오히려 불편하다, 사용하기 어렵다  
   - **측면 범주**: 편리성  
   - **감정 극성**: 부정적  

"""


### CHAT_LLM

In [13]:
# LLM 객체 생성
from langchain.chat_models import ChatOpenAI

from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [14]:
# LLM_2
chat_llm = ChatOpenAI(
    temperature=0.1,  # 창의성
    model_name="gpt-4o",  # 모델명
)

In [32]:
chat_template = """
You are an assistant for question-answering tasks.
The conversation has the following history:

{history}

Question:
{input}

Please answer in Korean.
"""

In [33]:
# 프롬프트 생성
chat_prompt = ChatPromptTemplate.from_template(chat_template)

In [34]:
from langchain.memory import ConversationBufferMemory #대화 기록을 "메모리에 저장" 하여 이후 대화에서 이전 대화의 내용을 참조할수 있게 함

memory = ConversationBufferMemory()
memory.save_context({"input": "Initial Information"}, {"output": ABSA_result})

In [35]:
from langchain.chains import ConversationChain  # 대화 기록을 바탕으로 연속적인 대화를 실행하는 체인

# 대화 체인 생성
conversation = ConversationChain(
    llm=chat_llm,
    memory=memory,
    prompt=chat_prompt,
)

In [36]:
# 대화 시작
while True:
    user_question = input("사용자의 질문을 입력하세요 (종료하려면 'exit' 입력): ")
    if user_question.lower() == 'exit':
        print("대화를 종료합니다.")
        break
    
    # 유저의 질문을 대화에 전달
    response = conversation({"input": user_question})
    print(response['response'])

/var/folders/m1/bqb2x_hs2p3c9xfm8sbxkgcr0000gn/T/ipykernel_26243/3536555042.py:9: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = conversation({"input": user_question})


{'input': '그래서 뭐가 더 제일 좋은거야?', 'history': 'Human: Initial Information\nAI: \n\n### 긍정적 요소\n1. **측면 용어**: 가격, 품질, 배송  \n   - **의견 용어**: 합리적, 만족스럽다, 좋다, 빠르다  \n   - **측면 범주**: 가격, 품질, 편리성  \n   - **감정 극성**: 긍정적  \n\n2. **측면 용어**: 효과, 사용  \n   - **의견 용어**: 기대 이상, 좋았다, 신뢰할 수 있다  \n   - **측면 범주**: 효과, 신뢰성  \n   - **감정 극성**: 긍정적  \n\n3. **측면 용어**: 성능, 효능, 신뢰성  \n   - **의견 용어**: 탁월하다, 믿음이 간다, 매우 만족한다  \n   - **측면 범주**: 성능, 신뢰성  \n   - **감정 극성**: 긍정적  \n\n### 중립적 요소\n1. **측면 용어**: 효과, 기대  \n   - **의견 용어**: 기대하고 있다, 아직 효과는 모르겠다  \n   - **측면 범주**: 효과, 기대  \n   - **감정 극성**: 중립적  \n\n2. **측면 용어**: 편리성, 사용  \n   - **의견 용어**: 나쁘지는 않다, 아직 결정할 수 없다  \n   - **측면 범주**: 편리성  \n   - **감정 극성**: 중립적  \n\n3. **측면 용어**: 품질, 기능  \n   - **의견 용어**: 무난하다, 보통이다, 기대만큼이다  \n   - **측면 범주**: 품질, 기능  \n   - **감정 극성**: 중립적  \n\n### 부정적 요소\n1. **측면 용어**: 효과, 기대  \n   - **의견 용어**: 기대만큼 효과가 없다, 효과가 미미하다  \n   - **측면 범주**: 효과  \n   - **감정 극성**: 부정적  \n\n2. **측면 용어**: 가격, 품질  \n   - **의견 용어**: 가격 대비 만족스럽지 않다, 별다른 효과가 없다  \n 

### Marketing